In [1]:
import os
import sys
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Install requirements
!pip install ood-metrics jsonargparse pytorch-lightning gitignore_parser
!pip install -r AnomalySegmentation/eomt/requirements.txt
!pip install "jsonargparse[signatures]"
!pip install lightning

# 3. Clone repo
REPO_URL = "https://github.com/MatteoSanna23/AnomalySegmentation.git"
REPO_DIR = "AnomalySegmentation"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'AnomalySegmentation/eomt/requirements.txt'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.5/760.5 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 61.7 MB/s eta 0:00:00


In [2]:
if not os.path.exists(REPO_DIR):
    print(">> Clonazione della repository...")
    !git clone --branch matte {REPO_URL}
else:
    print(">> Repository già presente.")

os.chdir(REPO_DIR)

>> Clonazione della repository...
Cloning into 'AnomalySegmentation'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 154 (delta 31), reused 22 (delta 17), pack-reused 89 (from 3)
Receiving objects: 100% (154/154), 26.89 MiB | 13.32 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [7]:
# 4. Preparazione Dataset (METODO ISTANTANEO: SYMLINK)
DATA_ROOT = "/content/data/cityscapes"
!mkdir -p {DATA_ROOT}

# Percorsi su Drive
ZIP_IMG_SOURCE = "/content/drive/MyDrive/Colab Notebooks/leftImg8bit_trainvaltest.zip"
ZIP_GT_SOURCE = "/content/drive/MyDrive/Colab Notebooks/gtFine_trainvaltest.zip"

# Percorsi Locali
ZIP_IMG_DEST = os.path.join(DATA_ROOT, "leftImg8bit_trainvaltest.zip")
ZIP_GT_DEST = os.path.join(DATA_ROOT, "gtFine_trainvaltest.zip")

print(">> Creazione collegamenti ai dataset...")

# Creiamo il collegamento simbolico invece di copiare
if os.path.exists(ZIP_IMG_SOURCE):
    # -s crea il link, -f forza la sovrascrittura se esiste già un file rotto
    !ln -sf "{ZIP_IMG_SOURCE}" "{ZIP_IMG_DEST}"
    print(f">> Link creato per le immagini.")
else:
    print("!! ERRORE: Non trovo il file zip delle immagini su Drive!")

if os.path.exists(ZIP_GT_SOURCE):
    !ln -sf "{ZIP_GT_SOURCE}" "{ZIP_GT_DEST}"
    print(f">> Link creato per le annotazioni.")
else:
    print("!! ERRORE: Non trovo il file zip delle annotazioni su Drive!")

print(">> Contenuto cartella dati (ora dovresti vedere i file in azzurrino/link):")
!ls -lh {DATA_ROOT}

>> Creazione collegamenti ai dataset...
>> Link creato per le immagini.
>> Link creato per le annotazioni.
>> Contenuto cartella dati (ora dovresti vedere i file in azzurrino/link):
total 20K
drwxr-xr-x 4 root root 4.0K Jan 19 10:53 gtFine
lrwxrwxrwx 1 root root   62 Jan 19 11:10 gtFine_trainvaltest.zip -> '/content/drive/MyDrive/Colab Notebooks/gtFine_trainvaltest.zip'
lrwxrwxrwx 1 root root   67 Jan 19 11:10 leftImg8bit_trainvaltest.zip -> '/content/drive/MyDrive/Colab Notebooks/leftImg8bit_trainvaltest.zip'
-rw-r--r-- 1 root root 1.7K Feb 17  2016 license.txt
-rw-r--r-- 1 root root  298 Feb 20  2016 README


In [9]:
!ls

eomt  eval  README.md  trained_models  wandb


In [13]:
# os.chdir("/content/AnomalySegmentation")

print(">>> Avvio del Training")
!touch .gitignore
# Note:
# - fit: è il comando obbligatorio
# - config: punta al file base_640 che è più leggero per Colab
# - data.init_args.path: dice al codice dove abbiamo scompattato i dati
# - trainer.max_epochs: metto 2 epoche per testare se tutto funziona velocemente
# --data.init_args.batch_size 1: Fondamentale per non finire la memoria GPU
# --trainer.accumulate_grad_batches 4: Simula un batch size più grande
# --compile_disabled: Risparmia memoria e tempo di avvio disattivando torch.compile

!python eomt/main.py fit \
    --config eomt/configs/dinov2/cityscapes/semantic/eomt_base_640.yaml \
    --data.init_args.path "{DATA_ROOT}" \
    --data.init_args.num_workers 2 \
    --data.init_args.batch_size 1 \
    --trainer.accumulate_grad_batches 4 \
    --trainer.max_epochs 1 \
    --trainer.limit_train_batches 50 \
    --trainer.limit_val_batches 10 \
    --trainer.accelerator gpu \
    --trainer.devices 1 \
    --compile_disabled

>>> Avvio del Training
/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
2026-01-19 11:39:30.500539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768822770.536094   15103 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory 

In [ ]:
'''
loss_file_content =
# --------------------------------------------------------------
# © 2025 Mobile Perception Systems Lab at TU/e. All rights reserved.
# Licensed under the MIT License.
# --------------------------------------------------------------

from typing import List, Optional
import torch.distributed as dist
import torch
import torch.nn as nn
import torch.nn.functional as F  # needed for F.normalize
from transformers.models.mask2former.modeling_mask2former import (
    Mask2FormerLoss,
    Mask2FormerHungarianMatcher,
)

class MaskClassificationLoss(Mask2FormerLoss):
    def __init__(
        self,
        num_points: int,
        oversample_ratio: float,
        importance_sample_ratio: float,
        mask_coefficient: float,
        dice_coefficient: float,
        class_coefficient: float,
        num_labels: int,
        no_object_coefficient: float,
        logit_norm_tau: float = 0.1, #! Update 1 : added parameter tau
    ):
        nn.Module.__init__(self)
        self.num_points = num_points
        self.oversample_ratio = oversample_ratio
        self.importance_sample_ratio = importance_sample_ratio
        self.mask_coefficient = mask_coefficient
        self.dice_coefficient = dice_coefficient
        self.class_coefficient = class_coefficient
        self.num_labels = num_labels
        self.eos_coef = no_object_coefficient

        self.logit_norm_tau = logit_norm_tau #! <--- Update 2: save Tau

        empty_weight = torch.ones(self.num_labels + 1)
        empty_weight[-1] = self.eos_coef
        self.register_buffer("empty_weight", empty_weight)

        self.matcher = Mask2FormerHungarianMatcher(
            num_points=num_points,
            cost_mask=mask_coefficient,
            cost_dice=dice_coefficient,
            cost_class=class_coefficient,
        )

    @torch.compiler.disable
    def forward(
        self,
        masks_queries_logits: torch.Tensor,
        targets: List[dict],
        class_queries_logits: Optional[torch.Tensor] = None,
    ):
        mask_labels = [
            target["masks"].to(masks_queries_logits.dtype) for target in targets
        ]
        class_labels = [target["labels"].long() for target in targets]

        #! <--- Update 3: LOGIT NORMALIZATION
        #! Normalize logits BEFORE they are used by the matcher or the loss.
        #! This forces the network to optimize cosine similarity.
        if class_queries_logits is not None:
            #! Calculate L2 norm along the last dimension (classes)
            norms = torch.norm(class_queries_logits, p=2, dim=-1, keepdim=True) + 1e-7
            #! Normalize and scale by temperature (1/tau)
            class_queries_logits = (class_queries_logits / norms) / self.logit_norm_tau

        #! Now pass the normalized logits to both the matcher and the loss function
        indices = self.matcher(
            masks_queries_logits=masks_queries_logits,
            mask_labels=mask_labels,
            class_queries_logits=class_queries_logits, #! Now normalized logits are used here
            class_labels=class_labels,
        )

        loss_masks = self.loss_masks(masks_queries_logits, mask_labels, indices)

        #! Also use normalized logits here
        loss_classes = self.loss_labels(class_queries_logits, class_labels, indices)

        return {**loss_masks, **loss_classes}

    def loss_masks(self, masks_queries_logits, mask_labels, indices):
        loss_masks = super().loss_masks(masks_queries_logits, mask_labels, indices, 1)

        num_masks = sum(len(tgt) for (_, tgt) in indices)
        num_masks_tensor = torch.as_tensor(
            num_masks, dtype=torch.float, device=masks_queries_logits.device
        )

        if dist.is_available() and dist.is_initialized():
            dist.all_reduce(num_masks_tensor)
            world_size = dist.get_world_size()
        else:
            world_size = 1

        num_masks = torch.clamp(num_masks_tensor / world_size, min=1)

        for key in loss_masks.keys():
            loss_masks[key] = loss_masks[key] / num_masks

        return loss_masks

    def loss_total(self, losses_all_layers, log_fn) -> torch.Tensor:
        loss_total = None
        for loss_key, loss in losses_all_layers.items():
            log_fn(f"losses/train_{loss_key}", loss, sync_dist=True)

            if "mask" in loss_key:
                weighted_loss = loss * self.mask_coefficient
            elif "dice" in loss_key:
                weighted_loss = loss * self.dice_coefficient
            elif "cross_entropy" in loss_key:
                weighted_loss = loss * self.class_coefficient
            else:
                raise ValueError(f"Unknown loss key: {loss_key}")

            if loss_total is None:
                loss_total = weighted_loss
            else:
                loss_total = torch.add(loss_total, weighted_loss)

        log_fn("losses/train_loss_total", loss_total, sync_dist=True, prog_bar=True)

        return loss_total  # type: ignore
'''

In [ ]:
'''
# Scriviamo il file nel percorso corretto
with open("eomt/training/mask_classification_loss.py", "w") as f:
    f.write(loss_file_content)

print(">> File mask_classification_loss.py aggiornato con la LogitNorm!")
'''